In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Create the dataset
data = [
    ("Show me all records from the users table", "SELECT", "*", None),
    ("Update the age of user with id 5 to 35", "UPDATE", "age = 35", "user_id = 5"),
    ("Add a new user with name 'Alice' and age 28", "INSERT INTO", "name, age", None),
    ("Delete the user with id 3", "DELETE FROM", None, "user_id = 3"),
    # Add more samples from the provided dataset
    ("List the names and ages from the employees table", "SELECT", "name, age", None),
    ("Get the email addresses of all customers", "SELECT", "email", None),
    ("Retrieve the product names and prices from the products table", "SELECT", "product_name, price", None),
    ("Fetch all data from the orders table where order_id is 102", "SELECT", "*", "order_id = 102"),
    ("Show all users who are older than 30", "SELECT", "*", "age > 30"),
    ("Change the status of order 200 to 'shipped'", "UPDATE", "status = 'shipped'", "order_id = 200"),
    ("Insert a new product with name 'Laptop' and price 999.99", "INSERT INTO", "product_name, price", None),
    ("Remove the product with id 15", "DELETE FROM", None, "product_id = 15"),
    # ...add the rest of the dataset
]

# Convert to DataFrame
df = pd.DataFrame(data, columns=["query", "intent", "columns", "conditions"])
df=pd.read_csv("output.csv")
# Fill None with empty string for easier processing
df.fillna('', inplace=True)

# Split the dataset into training and testing sets
X = df["query"]
y = df[["intent", "columns", "conditions"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [38]:
print(X_test)

13    Modify the email of customer with id 3 to 'new...
39         Remove employees from the finance department
30                            Delete the user with id 3
45    Update the contact number of customer with id ...
17    Change the address of customer 4 to '123 Main St'
48    Show the names and addresses of customers who ...
26    Add a new order with id 200 and total amount 5...
25    Insert a new product with id 12 and stock quan...
32                          Delete the order with id 80
19               Modify the discount of order 50 to 10%
Name: query, dtype: object


In [30]:
# Define a pipeline with TF-IDF vectorizer and a multi-output classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

# Train the model
pipeline.fit(X_train, y_train)


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', MultiOutputClassifier(estimator=LogisticRegression()))])

In [31]:
# Predict on the test set
y_pred = pipeline.predict(X_test)

# Print classification report for each output

print("Intent Classification Report:")
print(classification_report(y_test["intent"], y_pred[:, 0]))
print("\nColumns Classification Report:")
print(classification_report(y_test["columns"], y_pred[:, 1]))
print("\nConditions Classification Report:")
print(classification_report(y_test["conditions"], y_pred[:, 2]))


Intent Classification Report:
              precision    recall  f1-score   support

      DELETE       1.00      0.67      0.80         3
      INSERT       1.00      1.00      1.00         2
      SELECT       0.50      1.00      0.67         1
      UPDATE       1.00      1.00      1.00         4

    accuracy                           0.90        10
   macro avg       0.88      0.92      0.87        10
weighted avg       0.95      0.90      0.91        10


Columns Classification Report:
                                       precision    recall  f1-score   support

                                            0.38      1.00      0.55         3
                                    *       0.00      0.00      0.00         0
              address = '123 Main St'       0.00      0.00      0.00         1
      contact_number = '123-456-7890'       0.00      0.00      0.00         1
                        discount = 10       0.00      0.00      0.00         1
       email = 'newemail@exa

c:\Users\Anudeep Bura\.conda\envs\languageprocessing\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anudeep Bura\.conda\envs\languageprocessing\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anudeep Bura\.conda\envs\languageprocessing\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

In [39]:
i=0
print(X_test)
print(y_pred[i])

13    Modify the email of customer with id 3 to 'new...
39         Remove employees from the finance department
30                            Delete the user with id 3
45    Update the contact number of customer with id ...
17    Change the address of customer 4 to '123 Main St'
48    Show the names and addresses of customers who ...
26    Add a new order with id 200 and total amount 5...
25    Insert a new product with id 12 and stock quan...
32                          Delete the order with id 80
19               Modify the discount of order 50 to 10%
Name: query, dtype: object
['UPDATE' '' '']


In [ ]:
def predict_query_components(query):
    pred = pipeline.predict([query])
    return {
        'intent': pred[0, 0],
        'columns': pred[0, 1],
        'conditions': pred[0, 2]
    }

# Example usage
example_query = "Show me all student records"
components = predict_query_components(example_query)
print(f"Query: {example_query}")
print(f"Intent: {components['intent']}")
print(f"Columns: {components['columns']}")
print(f"Conditions: {components['conditions']}")


In [40]:
data=pd.read_csv("studentdatset.csv")
data.head()

,Query,Intent,Columns,Conditions
0,"Add a new student with name 'Alice', age 14, g...",INSERT,"(name, age, grade, section)","('Alice', 14, 9, 'A')"
1,Update attendance for student with id 3 to 95,UPDATE,attendance,id = 3
2,Get the names of all students in grade 10,SELECT,name,grade = 10
3,Delete student with id 5,DELETE,NaN,id = 5
4,"Add a new student with name 'Bob', age 15, gra...",INSERT,"(name, age, grade, section)","('Bob', 15, 10, 'B')"


In [42]:
data[[0]]

KeyError: "None of [Index([0], dtype='int32')] are in the [columns]"